In [9]:
from TableExtract import HTMLTableParser
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'http://www.cfbstats.com/2017/team/8/passing/defense/gamelog.html'

In [13]:
hp = HTMLTableParser()
df = hp.parse_url(url)[0][1]
df

,Date,Opponent,Surface,Result,Att,Comp,Pct.,Yards,Yards/Att,TD,Int,Rating
0,09/02/17,+ Florida St.,Turf,W 24-7,33.0,19.0,57.6,210.0,6.4,1.00,2.0,108.92
1,09/09/17,Fresno St.,Grass,W 41-10,41.0,26.0,63.4,216.0,5.3,1.00,1.0,110.84
2,09/16/17,Colorado St.,Grass,W 41-23,38.0,21.0,55.3,247.0,6.5,2.00,2.0,116.70
3,09/23/17,@ Vanderbilt,Turf,W 59-0,22.0,7.0,31.8,38.0,1.7,0.00,1.0,37.24
4,09/30/17,Mississippi,Grass,W 66-3,29.0,14.0,48.3,165.0,5.7,0.00,2.0,82.29
5,10/07/17,@ Texas A&M,Grass,W 27-19,29.0,19.0,65.5,237.0,8.2,1.00,1.0,138.64
6,10/14/17,Arkansas,Grass,W 41-9,43.0,23.0,53.5,200.0,4.7,1.00,1.0,95.59
7,10/21/17,Tennessee,Grass,W 45-7,16.0,9.0,56.3,44.0,2.8,0.00,1.0,66.85
8,11/04/17,18 LSU,Grass,W 24-10,31.0,15.0,48.4,155.0,5.0,0.00,1.0,83.94
9,11/11/17,@ 19 Mississippi St.,Grass,W 31-24,24.0,13.0,54.2,158.0,6.6,0.00,0.0,109.47


In [11]:
#FUNC: Drop last two rows
def dropTail(table):
    new_table = table.drop(table.tail(2).index)
    return new_table

In [14]:
#FUNC: Add game locations
def homeAway(table):
    game_locations = []

    for row in table['Opponent']:
        if '@' in row:
            game_locations.append('away')
        elif '+' in row:
            game_locations.append('neutral')
        else:
            game_locations.append('home')

    table['Location'] = game_locations
    return table

In [15]:
df = dropTail(df)
df

,Date,Opponent,Surface,Result,Att,Comp,Pct.,Yards,Yards/Att,TD,Int,Rating
0,09/02/17,+ Florida St.,Turf,W 24-7,33.0,19.0,57.6,210.0,6.4,1.0,2.0,108.92
1,09/09/17,Fresno St.,Grass,W 41-10,41.0,26.0,63.4,216.0,5.3,1.0,1.0,110.84
2,09/16/17,Colorado St.,Grass,W 41-23,38.0,21.0,55.3,247.0,6.5,2.0,2.0,116.70
3,09/23/17,@ Vanderbilt,Turf,W 59-0,22.0,7.0,31.8,38.0,1.7,0.0,1.0,37.24
4,09/30/17,Mississippi,Grass,W 66-3,29.0,14.0,48.3,165.0,5.7,0.0,2.0,82.29
5,10/07/17,@ Texas A&M,Grass,W 27-19,29.0,19.0,65.5,237.0,8.2,1.0,1.0,138.64
6,10/14/17,Arkansas,Grass,W 41-9,43.0,23.0,53.5,200.0,4.7,1.0,1.0,95.59
7,10/21/17,Tennessee,Grass,W 45-7,16.0,9.0,56.3,44.0,2.8,0.0,1.0,66.85
8,11/04/17,18 LSU,Grass,W 24-10,31.0,15.0,48.4,155.0,5.0,0.0,1.0,83.94
9,11/11/17,@ 19 Mississippi St.,Grass,W 31-24,24.0,13.0,54.2,158.0,6.6,0.0,0.0,109.47


In [16]:
df = homeAway(df)
df

,Date,Opponent,Surface,Result,Att,Comp,Pct.,Yards,Yards/Att,TD,Int,Rating,Location
0,09/02/17,+ Florida St.,Turf,W 24-7,33.0,19.0,57.6,210.0,6.4,1.0,2.0,108.92,neutral
1,09/09/17,Fresno St.,Grass,W 41-10,41.0,26.0,63.4,216.0,5.3,1.0,1.0,110.84,home
2,09/16/17,Colorado St.,Grass,W 41-23,38.0,21.0,55.3,247.0,6.5,2.0,2.0,116.70,home
3,09/23/17,@ Vanderbilt,Turf,W 59-0,22.0,7.0,31.8,38.0,1.7,0.0,1.0,37.24,away
4,09/30/17,Mississippi,Grass,W 66-3,29.0,14.0,48.3,165.0,5.7,0.0,2.0,82.29,home
5,10/07/17,@ Texas A&M,Grass,W 27-19,29.0,19.0,65.5,237.0,8.2,1.0,1.0,138.64,away
6,10/14/17,Arkansas,Grass,W 41-9,43.0,23.0,53.5,200.0,4.7,1.0,1.0,95.59,home
7,10/21/17,Tennessee,Grass,W 45-7,16.0,9.0,56.3,44.0,2.8,0.0,1.0,66.85,home
8,11/04/17,18 LSU,Grass,W 24-10,31.0,15.0,48.4,155.0,5.0,0.0,1.0,83.94,home
9,11/11/17,@ 19 Mississippi St.,Grass,W 31-24,24.0,13.0,54.2,158.0,6.6,0.0,0.0,109.47,away
